In [3]:
! pip install scipy --quiet
! pip install tenacity --quiet
! pip install tiktoken --quiet
! pip install termcolor --quiet
! pip install openai --quiet
! pip install tavily-python --quiet


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [25]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  
from dotenv import load_dotenv
from tavily import TavilyClient
from typing import Literal

load_dotenv()
GPT_MODEL = "gpt-4o"
EMBEDDING_MODEL = "text-embedding-3-small"
client = OpenAI()

In [26]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [27]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [30]:
def search_tool(query: str, max_results: int = 5, days: int = 1, topic: Literal['general', 'news'] = 'general'):
    """This tool provides access to Tavily search that can query internet"""
    search_depth = 'advanced'
    TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    response = tavily_client.search(query, max_results = 1, topic = 'general')
    results = response['results']
        

In [32]:
search_tool('Who is the president of United States of America')

[{'url': 'https://www.instagram.com/potus/?hl=en', 'title': 'President Donald J. Trump (@potus) • Instagram photos and videos', 'content': 'President Donald J. Trump 45th & 47th President of the United States. The Golden Age of America Begins Right Now.', 'score': 0.61196244, 'raw_content': None}]


In [17]:
SYSTEM_PROMPT = """
You are a helpful financial advisor assistant.
Respond to user's question by breaking it down into multiple queries that will you provide better context for the question.
Reason through each query, and provide a list of tools that will help you provide more context to answer the question.
A tool could be access to a function with finance related API endpoints, a google search, access to sec fillings, etc.
Your answer should be in the following format:
Question: question text
Query 1: Query 1 text
<reasoning>
</reasoning>
<tools>
1. tool 1
2. tool 2
...
n. tool n
</tools>
Query 2: Query 2 text
<reasoning>
</reasoning>
<tools>
1. tool 1
2. tool 2
...
n. tool n
</tools>
Query n: Query n text
<reasoning>
</reasoning>
<tools>
1. tool 1
2. tool 2
...
n. tool n
</tools>
"""
messages = []
messages.append({"role": "system", "content": SYSTEM_PROMPT})
messages.append({"role": "user", "content": "Is Google stock a good buy?"})
chat_response = chat_completion_request(messages)

chat_response.choices[0].message

ChatCompletionMessage(content="Question: Is Google stock a good buy?\n\nQuery 1: What is the current price of Google (Alphabet Inc.) stock?\n<reasoning>\nTo determine whether Google stock is a good buy, we need to start with its current market price. This provides context for evaluating value metrics such as price-to-earnings (P/E) ratio, historical price trends, and recent performance.\n</reasoning>\n<tools>\n1. Finance API to access current stock price data\n2. Stock market application or website (e.g., Yahoo Finance, Google Finance)\n</tools>\n\nQuery 2: What is Google's financial performance and recent earnings report?\n<reasoning>\nAnalyzing Google’s recent financial performance, including revenue, profit margins, and earnings per share, will help assess the company's financial health and growth potential. This involves understanding its capacity to generate profit and its performance relative to market expectations.\n</reasoning>\n<tools>\n1. Finance API for retrieving recent ear

For a given query, find out what alpha vantage endpoint provides the information needed to answer the given question.

In [3]:
# Fetch the Alpha Vantage API and store it in an embedding database. 
import chromadb
from bs4 import BeautifulSoup
import requests

In [5]:
# Fetch documentation
url = "https://www.alphavantage.co/documentation/"

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")
    sections = soup.find_all("h2")  # Assuming API sections are under h2 tags
    
    docs = {}
    for section in sections:
        title = section.get_text(strip=True)
        content = []
        
        for sibling in section.find_next_siblings():
            if sibling.name == "h2":
                break  # Stop when the next section starts
            content.append(sibling.get_text(strip=True))
        docs[title] = "\n".join(content)    
    
    with open('alpha_vantage_docs.txt', 'w', encoding='utf-8') as file:
        for title, content in docs.items():
            file.write(f"{title}\n{'-'*len(title)}\n{content}\n\n")

except requests.RequestException as e:
    print(f'Error fetching documentation: {e}')


In [ ]:
client = chromadb.Client()
collection = client.create_collection('alpha_vantage_endpoints')